In [1]:
from langchain.llms import LlamaCpp
from langchain.embeddings import LlamaCppEmbeddings

local_path = "../../models/game_npc_vicuna_huntress/ggml-f16.bin"
llm = LlamaCpp(model_path=local_path, verbose=True, n_batch=256, temperature=0.3, n_ctx=2048, use_mmap=False, stop=["###"])

llama.cpp: loading model from ../../models/game_npc_vicuna_huntress/ggml-f16.bin
llama_model_load_internal: format     = ggjt v1 (latest)
llama_model_load_internal: n_vocab    = 49954
llama_model_load_internal: n_ctx      = 2048
llama_model_load_internal: n_embd     = 4096
llama_model_load_internal: n_mult     = 256
llama_model_load_internal: n_head     = 32
llama_model_load_internal: n_layer    = 32
llama_model_load_internal: n_rot      = 128
llama_model_load_internal: ftype      = 1 (mostly F16)
llama_model_load_internal: n_ff       = 11008
llama_model_load_internal: n_parts    = 1
llama_model_load_internal: model size = 7B
llama_model_load_internal: ggml ctx size = 13448820.20 KB
llama_model_load_internal: mem required  = 14925.61 MB (+ 1026.00 MB per state)
....................................................................................................
llama_init_from_file: kv self size  = 1024.00 MB
AVX = 1 | AVX2 = 1 | AVX512 = 1 | AVX512_VBMI = 1 | AVX512_VNNI = 1 | FMA = 1 

In [2]:
from retrivers.llama_time_weighted_retriever import LlamaTimeWeightedVectorStoreRetriever
from vectorestores.chroma import EnhancedChroma

def create_new_memory_retriever():
    embeddings_model = LlamaCppEmbeddings(model_path=local_path)
    vs = EnhancedChroma(embedding_function=embeddings_model)
    return LlamaTimeWeightedVectorStoreRetriever(vectorstore=vs, other_score_keys=["importance"], k=14)


In [3]:
from generative_agents.llama_generative_agent import LlamaGenerativeAgent
from generative_agents.llama_memory import LlamaGenerativeAgentMemory

aoliweiya_memory = LlamaGenerativeAgentMemory(
    llm=llm,
    memory_retriever=create_new_memory_retriever(),
    reflection_threshold=8, # we will give this a relatively low number to show how reflection works
    verbose=True,
)

llama.cpp: loading model from ../../models/game_npc_vicuna_huntress/ggml-f16.bin
llama_model_load_internal: format     = ggjt v1 (latest)
llama_model_load_internal: n_vocab    = 49954
llama_model_load_internal: n_ctx      = 512
llama_model_load_internal: n_embd     = 4096
llama_model_load_internal: n_mult     = 256
llama_model_load_internal: n_head     = 32
llama_model_load_internal: n_layer    = 32
llama_model_load_internal: n_rot      = 128
llama_model_load_internal: ftype      = 1 (mostly F16)
llama_model_load_internal: n_ff       = 11008
llama_model_load_internal: n_parts    = 1
llama_model_load_internal: model size = 7B
llama_model_load_internal: ggml ctx size =  68.20 KB
llama_model_load_internal: mem required  = 14925.61 MB (+ 2052.00 MB per state)
llama_init_from_file: kv self size  =  512.00 MB
AVX = 1 | AVX2 = 1 | AVX512 = 1 | AVX512_VBMI = 1 | AVX512_VNNI = 1 | FMA = 1 | NEON = 0 | ARM_FMA = 0 | F16C = 1 | FP16_VA = 0 | WASM_SIMD = 0 | BLAS = 0 | SSE3 = 1 | VSX = 0 | 
Using 

In [4]:
aoliweiya = LlamaGenerativeAgent(
    name="奥莉薇娅",
    age=25,
    traits="坚强, 干练, 大姐姐", # You can add more persistent traits here
    status="导师", # When connected to a virtual world, we can have the characters update their status
    memory_retriever=create_new_memory_retriever(),
    llm=llm,
    memory=aoliweiya_memory,
    verbose=True,
)

llama.cpp: loading model from ../../models/game_npc_vicuna_huntress/ggml-f16.bin
llama_model_load_internal: format     = ggjt v1 (latest)
llama_model_load_internal: n_vocab    = 49954
llama_model_load_internal: n_ctx      = 512
llama_model_load_internal: n_embd     = 4096
llama_model_load_internal: n_mult     = 256
llama_model_load_internal: n_head     = 32
llama_model_load_internal: n_layer    = 32
llama_model_load_internal: n_rot      = 128
llama_model_load_internal: ftype      = 1 (mostly F16)
llama_model_load_internal: n_ff       = 11008
llama_model_load_internal: n_parts    = 1
llama_model_load_internal: model size = 7B
llama_model_load_internal: ggml ctx size =  14.06 KB
llama_model_load_internal: mem required  = 4984.75 MB (+ 2052.00 MB per state)
error loading model: llama.cpp: tensor 'layers.6.attention_norm.weight' is missing from model
llama_init_from_file: failed to load model


NameError: Could not load Llama model from path: ../../models/game_npc_vicuna_huntress/ggml-f16.bin

In [ ]:
print(aoliweiya.get_summary(force_refresh=True))

In [ ]:
# We can add memories directly to the memory object
aoliweiya_observations = [
    "猎人队长通过试训",
    "雪莉加入队伍",
    "妮可前来报名，但是晕倒在帐篷外",
    "艾丽娜怀疑妮可的能力，不希望她加入猎人小队",
    "猎人队长正在招聘新队员",
    "城外发现三只四级龙兽",
    "雪莉希望出城扫荡龙兽",
]
for observation in aoliweiya_observations:
    aoliweiya_memory.add_memory(observation)

In [ ]:
# Now that Aoliweiya has 'memories', their self-summary is more descriptive, though still rudimentary.
# We will see how this summary updates after more observations to create a more rich description.
print(aoliweiya.get_summary(force_refresh=True))

In [ ]:
print(aoliweiya.generate_reaction("猎人队长申请出城扫荡龙兽"))

In [ ]:
aoliweiya.generate_dialogue("领队大人", "你的小队准备好了吗？")